# Deploy a dask cluster with AWS cloudformation

## Objectives

Create a deploy a dask cluster using AWS cloud formation (success see file blabla). The current template creates:
- 1 dask-scheduler and 3 dask-workers. 
- A security group that allows a local machine to connect to the cluster (see the other notebook)
- EC2 role and connects this all instances to allow S3 access.
- S3 bucket in which we can store csv files

Things you have to do manually:
- upload the csv files to the s3 bucket
- create a user with programmatic access and configure the aws cli to work with these keys.

### TODO
- scheduler can be t2.micro but I want workers to be larger.
- use parameters to select the instance type for the scheduler and worker.
- To have 3 workers template of one is copied 3 times. It is better to maybe use auto scaling groups and define the number workers

### CloudFormation Template

execute the cell below if you want to see the entire cloud formation template in the notebook

In [19]:
%load cf-dask-cluster.yaml

You can use this template to create a CloudFormation stack on AWS. 

Once the stack is completed and all EC2 instances are running you can take the public IP of the scheduler and copy this in the code snippet below.

Everything should work smoothly (for me it did)

In [2]:
from dask.distributed import Client

# copy the public ip of scheduler from the output section of cloudformation
ip = "3.92.164.58"
address = f"tcp://{ip}:8786"
dashboard = f"http://{ip}:8787/status"

print(f"Use the link below to connect to the cluster dashboard:\n{dashboard}")

print(address)
client = Client(address=address)

client

Use the link below to connect to the cluster dashboard:
http://3.92.164.58:8787/status
tcp://3.92.164.58:8786


C:\Users\sacha\anaconda3\envs\dask-cluster\lib\site-packages\distributed\client.py:1606: VersionMismatchWarning: Mismatched versions found

+-------------+--------+-----------+---------+
| Package     | Client | Scheduler | Workers |
+-------------+--------+-----------+---------+
| cloudpickle | 3.0.0  | 3.1.0     | 3.1.0   |
+-------------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


<Client: 'tcp://172.31.47.163:8786' processes=3 threads=6, memory=11.46 GiB>

In [3]:
import dask.array as da

a_da = da.ones(10, chunks=5)
a_da

dask.array<ones_like, shape=(10,), dtype=float64, chunksize=(5,), chunktype=numpy.ndarray>

In [4]:
a_da_sum = a_da.sum()
a_da_sum

dask.array<sum-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

In [5]:
a_da_sum.compute()

np.float64(10.0)

In [6]:
xd = da.random.normal(10, 0.1, size=(30_000, 30_000), chunks=(3000, 3000))
xd

dask.array<normal, shape=(30000, 30000), dtype=float64, chunksize=(3000, 3000), chunktype=numpy.ndarray>

In [7]:
%%time
xd = da.random.normal(10, 0.1, size=(30_000, 30_000), chunks=(3000, 3000))
yd = xd.mean(axis=0)
yd.compute()

CPU times: total: 0 ns
Wall time: 5.04 s


array([10.00072162, 10.0000227 , 10.00135155, ...,  9.99986303,
        9.99949336,  9.99998057])

In [8]:
!aws s3 ls

2024-10-11 18:26:55 cf-templates-r78qtn9pvstf-us-east-1
2024-10-11 20:32:12 dask-input-data


In [10]:
import dask.dataframe as dd

# Read all CSV files from the root of the bucket
ddf = dd.read_csv("s3://dask-input-data/*.csv", 
                  dtype={"TailNum": str, "CRSElapsedTime": float, "Cancelled": bool},
                  blocksize="25MB" )


ddf

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,Diverted
npartitions=10,,,,,,,,,,,,,,,,,,,,,,,
,int64,int64,int64,int64,float64,int64,float64,int64,string,int64,string,float64,float64,float64,float64,float64,string,string,float64,float64,float64,bool,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [23]:
%%time
len(ddf)

CPU times: total: 0 ns
Wall time: 2.66 s


2611892

In [24]:
%%time
ddf.head(2)

CPU times: total: 78.1 ms
Wall time: 2.52 s


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,Diverted
0,1990,1,1,1,1621.0,1540,1747.0,1701,US,33,...,NaN,46.0,41.0,EWR,PIT,319.0,NaN,NaN,False,0
1,1990,1,2,2,1547.0,1540,1700.0,1701,US,33,...,NaN,-1.0,7.0,EWR,PIT,319.0,NaN,NaN,False,0


In [25]:
%%time
result = ddf.DepDelay.max()
result.compute()

CPU times: total: 15.6 ms
Wall time: 2.05 s


np.float64(1435.0)

In [26]:
%%time
len(ddf[~ddf.Cancelled])

CPU times: total: 31.2 ms
Wall time: 4.92 s


2540961

In [15]:
%%time
ddf[~ddf.Cancelled].groupby("Origin")["Origin"].count().compute()

CPU times: total: 15.6 ms
Wall time: 2.92 s


Origin
EWR    1139451
JFK     427243
LGA     974267
Name: Origin, dtype: int64

In [21]:
%%time
ddf.groupby("Origin").DepDelay.mean().compute()

CPU times: total: 31.2 ms
Wall time: 2.74 s


Origin
EWR    10.295469
JFK    10.351299
LGA     7.431142
Name: DepDelay, dtype: float64

In [22]:
%%time
ddf.groupby("DayOfWeek").DepDelay.mean().idxmax().compute()

CPU times: total: 31.2 ms
Wall time: 2.02 s


np.int64(5)

In [27]:
client.close()